In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import dateutil
from sklearn import model_selection
from sklearn.linear_model import LinearRegression

In [23]:
dataset = pd.read_csv('ar.csv')
print(dataset.columns.values)

['countryCode' 'customerID' 'PaperlessDate' 'invoiceNumber' 'InvoiceDate'
 'DueDate' 'InvoiceAmount' 'Disputed' 'SettledDate' 'PaperlessBill'
 'DaysToSettle' 'DaysLate']


In [24]:
#total no of invoices
totalinvoices=dataset.shape[0]
#dataset['customerID'].count()
print("Total no of invoice:",totalinvoices)

Total no of invoice: 2466


In [25]:
#No. of delayed invoices
delayed=dataset[(dataset.DaysLate>0)]
totaldelayed=delayed.shape[0]
print("No of delayed invoices:",totaldelayed)

No of delayed invoices: 877


In [26]:
percentofdelayed=(totaldelayed/totalinvoices)*100
print("percentage of delayed invoice:",percentofdelayed)

percentage of delayed invoice: 35.56366585563666


In [27]:
#No of payers in the data set
#totalpayers=len(dataset.groupby('customerID'))
#print(dataset['customerID'].value_counts())
totalpayers=(dataset['customerID'].nunique())
print("Total no of payers:",totalpayers)

Total no of payers: 100


In [28]:
# max delayed days
print("max delayed days:",dataset['DaysLate'].max())

max delayed days: 45


In [29]:
#No of invoice per payer
grouped=dataset.groupby('customerID',as_index=False)
invoice_count=grouped.agg({"invoiceNumber":"count"})
invoice_count.columns=['customerID','total']
invoice_count_sort=invoice_count.sort_values('customerID')
#print(left.loc['0379-NEVHP'])
print(invoice_count.shape)
print(invoice_count.columns.values)
print(invoice_count_sort.head(5))

#my.plot()
#plt.show()

(100, 2)
['customerID' 'total']
   customerID  total
0  0187-ERLSR     16
1  0379-NEVHP     27
2  0465-DTULQ     26
3  0625-TNJFG     28
4  0688-XNJRO     34


In [30]:
# delayed invoices per payer
delayed=delayed.groupby('customerID',as_index=False)
delayed_invoice_count=delayed.agg({'invoiceNumber':'count'})
#right = pd.DataFrame(mydelay)
delayed_invoice_count.columns=['customerID','delayed']

print(delayed_invoice_count.shape)
delayed_invoice_count_sort=delayed_invoice_count.sort_values('customerID')
print(delayed_invoice_count.columns.values)
print(delayed_invoice_count_sort.head(5))
#mydelay.plot()
#plt.show()

(83, 2)
['customerID' 'delayed']
   customerID  delayed
0  0379-NEVHP        1
1  0465-DTULQ       14
2  0625-TNJFG        5
3  0688-XNJRO       32
4  0706-NRGUP       10


In [31]:
delayed_days_avg=delayed.agg({'DaysLate':'mean'})
delayed_days_avg.columns=['customerID','avgDaysDelayed']
print(delayed_days_avg.head(5))


   customerID  avgDaysDelayed
0  0379-NEVHP       17.000000
1  0465-DTULQ        9.857143
2  0625-TNJFG        4.000000
3  0688-XNJRO       15.343750
4  0706-NRGUP        6.900000


In [32]:
settled_days_avg=grouped.agg({'DaysToSettle':'mean'})
settled_days_avg.columns=['customerID','avgDaysToSettle']
print(settled_days_avg.head(5))

   customerID  avgDaysToSettle
0  0187-ERLSR        12.937500
1  0379-NEVHP        17.444444
2  0465-DTULQ        33.730769
3  0625-TNJFG        24.750000
4  0688-XNJRO        44.382353


In [33]:
# delay ratio per payer
invoice_count_stats=pd.merge(invoice_count,delayed_invoice_count,on='customerID',how='left').fillna(0)
invoice_count_stats=invoice_count_stats.sort_values('customerID')
invoice_count_stats['paid']=invoice_count_stats['total']-invoice_count_stats['delayed']
invoice_count_stats['delayRatio']=(invoice_count_stats['delayed']/invoice_count_stats['paid'])*100
print(invoice_count_stats.head(10))

   customerID  total  delayed  paid   delayRatio
0  0187-ERLSR     16      0.0  16.0     0.000000
1  0379-NEVHP     27      1.0  26.0     3.846154
2  0465-DTULQ     26     14.0  12.0   116.666667
3  0625-TNJFG     28      5.0  23.0    21.739130
4  0688-XNJRO     34     32.0   2.0  1600.000000
5  0706-NRGUP     18     10.0   8.0   125.000000
6  0709-LZRJV     25     11.0  14.0    78.571429
7  0783-PEPYR     21     20.0   1.0  2000.000000
8  1080-NDGAE     31     18.0  13.0   138.461538
9  1168-BEASA     23      2.0  21.0     9.523810


In [34]:
# total paid invoice amount per payer
paid_tot=grouped.agg({"InvoiceAmount":"sum"})
print(paid_tot.columns.values)
paid_tot.columns=['customerID','totalAmt']
print(paid_tot.head(5))

['customerID' 'InvoiceAmount']
   customerID  totalAmt
0  0187-ERLSR   1072.63
1  0379-NEVHP   1584.18
2  0465-DTULQ   1360.12
3  0625-TNJFG   1627.26
4  0688-XNJRO   1231.45


In [35]:
# total delayed invoice amount per payer
delayed_tot=delayed.agg({"InvoiceAmount":"sum"})
delayed_tot.columns=['customerID','delayedAmt']
print(delayed_tot.head(5))

   customerID  delayedAmt
0  0379-NEVHP       48.65
1  0465-DTULQ      670.39
2  0625-TNJFG      301.94
3  0688-XNJRO     1181.00
4  0706-NRGUP      325.42


In [36]:
invoice_amt_stats=pd.merge(paid_tot,delayed_tot,on='customerID',how='left').fillna(0)
invoice_amt_stats['paidamt']=invoice_amt_stats['totalAmt']-invoice_amt_stats['delayedAmt']
invoice_amt_stats['delayAmtRatio']=(invoice_amt_stats['delayedAmt']/invoice_amt_stats['totalAmt'])*100
print(invoice_amt_stats.head(5))

   customerID  totalAmt  delayedAmt  paidamt  delayAmtRatio
0  0187-ERLSR   1072.63        0.00  1072.63       0.000000
1  0379-NEVHP   1584.18       48.65  1535.53       3.070989
2  0465-DTULQ   1360.12      670.39   689.73      49.289033
3  0625-TNJFG   1627.26      301.94  1325.32      18.555117
4  0688-XNJRO   1231.45     1181.00    50.45      95.903204


In [37]:
payer_stats=pd.merge(invoice_count_stats,invoice_amt_stats,on="customerID",how='left')
payer_stats=pd.merge(payer_stats,delayed_days_avg,on="customerID",how="left").fillna(0)
payer_stats=pd.merge(payer_stats,settled_days_avg,on="customerID",how="left").fillna(0)


print(payer_stats.head(2))


   customerID  total  delayed  paid  delayRatio  totalAmt  delayedAmt  \
0  0187-ERLSR     16      0.0  16.0    0.000000   1072.63        0.00   
1  0379-NEVHP     27      1.0  26.0    3.846154   1584.18       48.65   

   paidamt  delayAmtRatio  avgDaysDelayed  avgDaysToSettle  
0  1072.63       0.000000             0.0        12.937500  
1  1535.53       3.070989            17.0        17.444444  


In [55]:
#dataset.columns.values[1]='payer'
#dataset_new=dataset.join(payer_stats.set_index('payer'))
dataset_new=pd.merge(dataset,payer_stats,on='customerID',how='left')
#dataset_new.columns.values
#print(dataset_new.head(1))
dataset_new=dataset_new[['customerID', 'InvoiceAmount','total','delayRatio','totalAmt','delayAmtRatio','avgDaysDelayed','DaysLate']]
#print(dataset_new.columns.values)
#print(dataset_new.shape)
dummies = pd.get_dummies(dataset_new.customerID)
print(dataset_new.head(1))


   customerID  InvoiceAmount  total  delayRatio  totalAmt  delayAmtRatio  \
0  0379-NEVHP          55.94     27    3.846154   1584.18       3.070989   

   avgDaysDelayed  DaysLate  
0            17.0         0  


In [48]:
array=dataset_new.values
X=array[:,0:7]
Y=array[:,7]
validation_size=0.20
seed=7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X,Y,test_size=validation_size,random_state=seed)

In [49]:
lm=LinearRegression()
lm.fit(X_train,Y_train)

ValueError: could not convert string to float: '5920-DPXLN'